In [11]:
import csv
import pandas as pd
import json
import random

In [12]:

data = []
with open('C:\\Users\\LENOVO\\PycharmProjects\\Ayurvedic_HIS\\Kartik Aslia\\ona-HIS-Ayu\\Data\\Labeled_Datasets\\ADDH\\Drug prescription Dataset.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append(row)

In [13]:
converted_data = []
for row in data:
    age = row[1]
    disease = row[0]
    severity = row[3]
    drug = row[4]
    formatted_string = f"###user: my 'age' is {age} and I am suffering from {disease}, severity is '{severity}', ###assistant: you must have '{drug}' it will help you"
    converted_data.append(formatted_string)


In [14]:
# Shuffle the data randomly
random.shuffle(converted_data)

# Calculate the split index
split_index = int(0.8 * len(converted_data))

# Split the data into train and eval sets
train_data = converted_data[:split_index]
eval_data = converted_data[split_index:]

# Save train data to JSON file
with open('train_data.json', 'w') as train_file:
    json.dump(train_data, train_file)

# Save eval data to JSON file
with open('eval_data.json', 'w') as eval_file:
    json.dump(eval_data, eval_file)


In [15]:

df = pd.DataFrame(converted_data)
df.to_csv('output.txt', index=False, header=False)


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Load the train and evaluation datasets
train_dataset = load_dataset('json', data_files='notes.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='notes_validation.jsonl', split='train')

# Create a formatting function to structure training examples as prompts
def formatting_func(example):
    text = f"### Question: {example['input']}\n### Answer: {example['output']}"
    return [text]

# Initialize the tokenizer and model
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Format the datasets using the formatting function
train_dataset = train_dataset.map(formatting_func, batched=True)
eval_dataset = eval_dataset.map(formatting_func, batched=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Fine-tune the model
trainer.train()

# Save the model
trainer.save_model("fine_tuned_model")

# Load the model
model = AutoModelForCausalLM.from_pretrained("fine_tuned_model")
